# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [71]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [72]:
weather_data = pd.read_csv ('weather_data.csv')
del weather_data["Unnamed: 0"]
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,20.43,76,11,3.24,US,1644204868
1,Elizabeth City,36.2946,-76.2511,41.05,84,100,10.36,US,1644205049
2,Cayenne,4.9333,-52.3333,80.10,87,75,3.00,GF,1644205119
3,Cockburn Town,21.4612,-71.1419,75.81,77,0,15.39,TC,1644204907
4,Port Alfred,-33.5906,26.8910,72.93,79,82,4.97,ZA,1644204555
...,...,...,...,...,...,...,...,...,...
566,Half Moon Bay,37.4636,-122.4286,59.25,72,100,9.22,US,1644205342
567,Bonfim,3.0833,-59.9500,77.00,79,94,7.14,BR,1644205342
568,Bilibino,68.0546,166.4372,-12.50,77,18,4.41,RU,1644205343
569,Palana,59.1167,159.9667,14.22,77,67,8.90,RU,1644205343


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [73]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [74]:
# Store latitude and longitude in locations
location = weather_data[["Lat", "Lng"]]
humidity = weather_data['Humidity']

In [75]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# # Customize the size of the figure
# figure_layout = {
#     'width': '400px',
#     'height': '300px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# }
# fig = gmaps.figure(layout=figure_layout)
fig
#ask gretel why there are multiple earths

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [76]:
# Fill NaN values
weather_data = weather_data.dropna()
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Albany,42.6001,-73.9662,20.43,76,11,3.24,US,1644204868
1,Elizabeth City,36.2946,-76.2511,41.05,84,100,10.36,US,1644205049
2,Cayenne,4.9333,-52.3333,80.10,87,75,3.00,GF,1644205119
3,Cockburn Town,21.4612,-71.1419,75.81,77,0,15.39,TC,1644204907
4,Port Alfred,-33.5906,26.8910,72.93,79,82,4.97,ZA,1644204555
...,...,...,...,...,...,...,...,...,...
566,Half Moon Bay,37.4636,-122.4286,59.25,72,100,9.22,US,1644205342
567,Bonfim,3.0833,-59.9500,77.00,79,94,7.14,BR,1644205342
568,Bilibino,68.0546,166.4372,-12.50,77,18,4.41,RU,1644205343
569,Palana,59.1167,159.9667,14.22,77,67,8.90,RU,1644205343


In [85]:
weather_data_new = weather_data.loc[((weather_data['Max Temp'] <= 80) & (weather_data['Max Temp'] >= 70)) & ((weather_data['Wind Speed'] < 10) & (weather_data['Cloudiness'] == 0))]
hotel_df = weather_data_new
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
158,East London,-33.0153,27.9116,70.74,94,0,4.61,ZA,1644205180
352,Port Hueneme,34.1478,-119.1951,74.32,43,0,4.61,US,1644205257
383,La Palma,33.8464,-118.0467,74.21,41,0,4.61,US,1644205270
387,Porto Velho,-8.7619,-63.9039,73.45,94,0,3.44,BR,1644205272
487,Cabedelo,-6.9811,-34.8339,79.47,74,0,8.95,BR,1644205312
511,Natitingou,10.3042,1.3796,75.25,20,0,7.38,BJ,1644205321


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [78]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
158,East London,-33.0153,27.9116,70.74,94,0,4.61,ZA,1644205180
352,Port Hueneme,34.1478,-119.1951,74.32,43,0,4.61,US,1644205257
383,La Palma,33.8464,-118.0467,74.21,41,0,4.61,US,1644205270
387,Porto Velho,-8.7619,-63.9039,73.45,94,0,3.44,BR,1644205272
487,Cabedelo,-6.9811,-34.8339,79.47,74,0,8.95,BR,1644205312
511,Natitingou,10.3042,1.3796,75.25,20,0,7.38,BJ,1644205321


In [79]:
hotel_names = []
lat = hotel_df['Lat']
lng = hotel_df['Lng']
lat_lngs = zip(lat,lng)

for lat_lng in lat_lngs:

    target_coordinates = f"{lat_lng[0]},{lat_lng[1]}"
    target_radius = 5000
    target_type = "hotel"


    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "types": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
    
   
    places_data = response.json()

    print(json.dumps(places_data, indent=4, sort_keys=True))

    try:
        hotel_names.append(places_data["results"][1]["name"])
    except:
        print(f"No hotel found for {lat_lng}.")
        hotel_names.append("")

{
    "html_attributions": [],
    "next_page_token": "Aap_uEC81gb7c_ochrBz29k6is5EbAeSoX4Xm_kNcaEv6gm0fsH2E47KTbLGZKI1sb3LHVZU-wuGvxTrj-5r5crRBOEXUjNnLjxWGCqHLBXEei7X18pUGHGwW4xe8FHrdQlefFkJdYAH-OBHEZrXZQkuvjZfs1QZ1hwXrUoFi4lKjis-pGItZ20kAuuaLRG-5vkBKGRNauQVy0nEFIrPja3qCGlyMOVnBc6PcDDtMoE817Fc5XLwAFyaX11XJL-pIVtcWhvou8ZiTex3VZEAfdE10Q3AreQV8h9BGleQIxmRxQzrngdp0fc9Dl2cn9JdC2IWlI1S8Ms9V3L91CrKQ1QlrdXrg9DBrJUFVgJCRWtzZSjphu3mIDfV4tptASuNNJ1BRqYGIBXG9rXLx6kL2yE_-Lihp38wWX2QmNgGa0-O1HJWQFs_U2K_gQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -33.0291582,
                    "lng": 27.8545867
                },
                "viewport": {
                    "northeast": {
                        "lat": -32.83636093326298,
                        "lng": 28.03534503287705
                    },
                    "southwest": {
                        "lat": -33.07490888355496,
                        "lng": 27.525842946485

In [80]:
hotel_names

['Tu Casa',
 'Hampton Inn Channel Islands Harbor',
 'La Quinta Inn & Suites by Wyndham Buena Park',
 'Hotel Central',
 'IFPB - Campus Cabedelo',
 'Ma Résistance']

In [81]:
hotel_df["Hotel Name"] = hotel_names
hotel_df


There was no hotel located within 5000m of Adrar.


C:\Users\marqu\AppData\Local\Temp/ipykernel_26284/4052694212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = hotel_names


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
158,East London,-33.0153,27.9116,70.74,94,0,4.61,ZA,1644205180,Tu Casa
352,Port Hueneme,34.1478,-119.1951,74.32,43,0,4.61,US,1644205257,Hampton Inn Channel Islands Harbor
383,La Palma,33.8464,-118.0467,74.21,41,0,4.61,US,1644205270,La Quinta Inn & Suites by Wyndham Buena Park
387,Porto Velho,-8.7619,-63.9039,73.45,94,0,3.44,BR,1644205272,Hotel Central
487,Cabedelo,-6.9811,-34.8339,79.47,74,0,8.95,BR,1644205312,IFPB - Campus Cabedelo
511,Natitingou,10.3042,1.3796,75.25,20,0,7.38,BJ,1644205321,Ma Résistance


In [82]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [83]:
# Add marker layer on top of heat map
fig = gmaps.figure()

#create markers
markers = gmaps.marker_layer(locations)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#trying out different gradients
heat_layer.gradient = [
    'white',
    'purple',
    'blue'
]

#add heat layer
fig.add_layer(heat_layer)

#add marker layer
fig.add_layer(markers)

# Add info marker layer
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))